In [192]:
import numpy as np
import pandas as pd

train = pd.read_csv("train.csv")
test  = pd.read_csv("test.csv")

In [193]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 90.5+ KB


In [194]:
def clean_data(titanic):
    
    titanic["Age"] = titanic["Age"].fillna(titanic["Age"].median())
    titanic["Age"].median()
    
    titanic.loc[titanic["Sex"] == "male", "Sex"] = 0
    titanic.loc[titanic["Sex"] == "female", "Sex"] = 1
    
    titanic["Embarked"] = titanic["Embarked"].fillna("S")
    

    titanic.loc[titanic["Embarked"] == "S", "Embarked"] = 0
    titanic.loc[titanic["Embarked"] == "C", "Embarked"] = 1
    titanic.loc[titanic["Embarked"] == "Q", "Embarked"] = 2

    titanic["Fare"] = titanic["Fare"].fillna(titanic["Fare"].median())
    
    titanic["FamilySize"] = titanic['Parch'] + titanic['SibSp']

    return titanic

In [195]:


def create_submission(model, train, test, predictors, filename):

    model.fit(train[predictors], train["Survived"])
    predictions = model.predict(test[predictors])

    submission = pd.DataFrame({
        "PassengerId": test["PassengerId"],
        "Survived": predictions
    })
    
    submission.to_csv(filename, index=False)



In [196]:
train_data = clean_data(train)
test_data  = clean_data(test)

In [197]:
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,FamilySize
0,1,0,3,"Braund, Mr. Owen Harris",0,22,1,0,A/5 21171,7.2500,NaN,0,1
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,38,1,0,PC 17599,71.2833,C85,1,1
2,3,1,3,"Heikkinen, Miss. Laina",1,26,0,0,STON/O2. 3101282,7.9250,NaN,0,0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,35,1,0,113803,53.1000,C123,0,1
4,5,0,3,"Allen, Mr. William Henry",0,35,0,0,373450,8.0500,NaN,0,0


In [220]:


from sklearn.ensemble import RandomForestClassifier
from sklearn import cross_validation
from sklearn.ensemble import GradientBoostingClassifier

model = GradientBoostingClassifier(n_estimators=50,max_depth=800,min_samples_leaf=12,min_samples_split=14)

predictors = ["Pclass", "Sex", "Age", "FamilySize", "Fare", "Embarked"]

#model = RandomForestClassifier(
#    random_state=1,
#    n_estimators=1000,
#    min_samples_split=8,
#    min_samples_leaf=6
#)



scores = cross_validation.cross_val_score(
    model,
    train_data[predictors],
    train_data["Survived"],
    cv=3
)

print(scores.mean())

0.819304152637


In [199]:
create_submission(model, train_data, test_data, predictors, "submission.csv")